# Analiza zbioru danych związanych z katastrofą Titanica
### Na podsawie poniższej analizy chciałbym zobaczyć i określić czynniki związane z przeżywalonością pasażerów statku, oraz określić który z modeli predykcyjnych poradzi sobie z określeniem który z pasażerów ma największą szansę przeżycia

## 1. Określenie Problemu
### Zrozumienie wpływu czynników decydujących na przezprzywalności pasażerów podczas katastrofy Titanica.
* Czy istnieje różnica w szansach przeżycia między mężczyznami a kobietami?
* Czy klasa biletu związna była z większą szansą przeżycia?
* Czy wiek pasażera miał wpływ na jego szanse przeżycia?
* Czy inne czynniki, takie jak stan cywilny, wielkość rodziny, bądz miejsce na statku miały istone znaczenie.

## 2. Wstępne zapoznanie się z danymi 
### Załadowanie podstawowych bibliotek, oraz bazy danych.

In [217]:
#Obliczenia  i analiza  danych
import numpy as np
import pandas as pd

#wizualiacja 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set_palette('icefire_r', 2)


%matplotlib inline
#Przetwarzanie

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
# Baza danych gównych do nauki modelu
Titanic = pd.read_csv("Titanic_Database.csv") 
# Baza danych Titanica do sprawdzenie modelu
test = pd.read_csv("test.csv")

In [218]:
Titanic.head(8)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [219]:
print(Titanic.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


### Zbiór danych zawiera następujące informacje:

*  PassengerId: Unikatowy identyfikator pasażera
*  Survived: Czy pasażer przeżył (1) czy zginął (0)
*  Pclass: Klasa biletu (1=pierwsza, 2=druga, 3=trzecia)
*  Name: Imię i nazwisko pasażera, staus cywilny
*  Sex: Płeć pasażera (male, female)
*  Age: Wiek pasażera
*  SibSp: Liczba rodzeństwa i/lub małżonka na pokładzie
*  Parch: Liczba rodziców i/lub dzieci na pokładzie
*  Ticket: Numer biletu
*  Fare: Cena biletu
*  Cabin: Numer kabiny
*  Embarked: Port zaokrętowania (C = Cherbourg, Q = Queenstown, S = Southampton)

## 3. Czyszczenie i przygotowanie danych

In [220]:
#Sprawdzenie wartości zerowych dla głównej bazy danych
print(pd.isnull(Titanic).sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [221]:
print(pd.isnull(test).sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


#### W przypadku głownej bazy danych brakuje 177 kluczowych wartość dla parametru Age, natomiast w przypadku bazy testowej jest to 86 brakująchyc wartości. Kategoria Cabin zostanie w początkowej fazie testó usunięta.


### Encodowanie danych kategorycznych

#### Płeć.

In [222]:
encoder = OneHotEncoder(sparse_output=False, drop='first')

for df in [Titanic, test]:
    sex_encoded = encoder.fit_transform(df['Sex'].values.reshape(-1, 1))  # Dopasuj i przekształc jednocześnie
    df['Sex_Enc'] = sex_encoded

### Male = 1 ||| Female = 0 

#### Stan cywilny paseżerów.

In [223]:
def get_title(name):
    # Wyodrębnij prefix z imienia
    title = name.split(',')[1].split('.')[0].strip()
    if title in ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']:
        return 'Rare'
    elif title in ['Mlle']:
        return 'Miss'
    elif title in ['Ms']:
        return 'Miss'
    elif title in ['Mme']:
        return 'Mrs'
    else:
        return title

Titanic['Title'] = Titanic['Name'].apply(get_title)
test['Title']    = test['Name'].apply(get_title)
##
#title_percentages1 = Titanic['Title'].value_counts(normalize=True)
#title_percentages2 = test['Title'].value_counts(normalize=True)
#print(title_percentages1)
#print(title_percentages2)

In [224]:
title_mapping = {
    "Mr": 0,
    "Miss": 1,
    "Mrs": 2,
    "Master": 3,
    "Rare": 4,
    "the Countess": 5
}

for df in [Titanic, test]:
    df['Title_Enc'] = np.array([title_mapping[x] for x in df['Title']])

#title_percentages1 = Titanic['Title'].value_counts(normalize=True)
#title_percentages2 = test['Title'].value_counts(normalize=True)
#print(title_percentages1)
#print(title_percentages2)

#### Port zaokrętowania

In [225]:
title_mapping = {
    "C": 0,
    "Q": 1,
    "S": 2,
}

for df in [Titanic, test]:
    df['Embarked'] = pd.Categorical(df['Embarked'])
    df['Embarked_Enc'] = df['Embarked'].cat.codes


#### Uzupełnianie brakujących wartości
Znacząco wybiegając w przyszłość, ponieważ kolumny Age, Embarked i Fare posiadają brakujące wartości, więc wykorzystując dalej przedstawioną w tekście macierz korelacji perasona zastosuję zależne najmnocniej skolerowane z tymi kolumnami paramtery. 

In [226]:
# Utwórz imputer KNN
imputer = KNNImputer(n_neighbors=5)
# Dopasuj imputer do danych
for df in [Titanic, test]:
    imputer.fit(df[['Age', 'Pclass', 'SibSp', 'Parch', 'Fare']].dropna())
    df['Age'] = imputer.transform(df[['Age', 'Pclass', 'SibSp', 'Parch', 'Fare']])

In [227]:
# Utwórz imputer KNN
imputer = KNNImputer(n_neighbors=5)
# Dopasuj imputer do danych
for df in [Titanic, test]:
    imputer.fit(df[['Embarked_Enc' ,'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']].dropna())
    df['Embarked_Enc'] = imputer.transform(df[['Embarked_Enc', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']])

In [228]:
# Utwórz imputer KNN
imputer = KNNImputer(n_neighbors=5)
# Dopasuj imputer do danych
imputer.fit(test[['Fare', 'Age', 'Pclass', 'SibSp', 'Parch' ]].dropna())
df['Fare'] = imputer.transform(test[['Fare', 'Age', 'Pclass', 'SibSp', 'Parch' ]])

In [229]:
print(pd.isnull(Titanic).sum())

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin           687
Embarked          2
Sex_Enc           0
Title             0
Title_Enc         0
Embarked_Enc      0
dtype: int64


In [230]:
print(pd.isnull(test).sum())

PassengerId       0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin           327
Embarked          0
Sex_Enc           0
Title             0
Title_Enc         0
Embarked_Enc      0
dtype: int64


In [231]:
### Sumaryczne wyniki/punkt odniesnia dla danych kattegorycznych, kótóre zostały zencodowane. 

## Wybór danych
#### Ocena kategorii, które mogą zostać bezwzględnie usunięte ponieważ nie planuję uwzględniać ich dalszej części modelowania, bądz zostały zencodowane.   

In [233]:
# Usuń oryginalne kolumny
categorical_features = ['PassengerId','Name', 'Title', 'Sex', 'Ticket','Cabin', 'Embarked']
Titanic.drop(columns=categorical_features, inplace=True)
Titanic.describe()

KeyError: "['PassengerId', 'Name', 'Title', 'Sex', 'Ticket', 'Cabin', 'Embarked'] not found in axis"

In [ ]:
categorical_features = ['PassengerId','Name', 'Title', 'Sex', 'Ticket','Cabin', 'Embarked']
test.drop(columns=categorical_features, inplace=True)
test.descibe()

In [ ]:
Titanic[['Age', 'Fare']] = Titanic[['Age', 'Fare']].astype(np.float32)
columns_to_convert = ['Survived' ,'Pclass', 'SibSp', 'Parch', 'Sex_Enc', 'Embarked_Enc', 'Title_Enc']
Titanic[columns_to_convert] = Titanic[columns_to_convert].astype(np.int16)
Titanic.info()

In [ ]:
test[['Age', 'Fare']] = test[['Age', 'Fare']].astype(np.float32)
columns_to_convert = ['Pclass', 'SibSp', 'Parch', 'Sex_Enc', 'Embarked_Enc', 'Title_Enc']
test[columns_to_convert] = test[columns_to_convert].astype(np.int16)
test.info()

## 4. Wizualizacja i ekspoloracja wzorców

In [ ]:
Titanic.groupby('Sex_Enc').mean().round(2)

In [ ]:
test.groupby('Sex_Enc').mean().round(2)

In [ ]:
Titanic.groupby('Survived').mean().round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
sns.heatmap(Titanic.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, linecolor='white', annot=True, cmap= 'icefire', fmt=".2f")
ax.set_title('Pearson Correlation of Features', y=1.05, size=15)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
sns.heatmap(test.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, linecolor='white', annot=True, cmap= 'icefire', fmt=".2f")
ax.set_title('Pearson Correlation of Features', y=1.05, size=15)

In [ ]:
#sns.pairplot(Titanic, hue="Survived", kind="kde", corner=True)

###  Age distribution

In [ ]:
fig, axes = plt.subplots(figsize=(14,8))
axes.set_ylabel('Liczba pasażerów')
axes.set_xlabel('Age')
axes.set_title('Age distribution')
sns.histplot(Titanic[Titanic['Survived']==0]['Age'],kde=True, bins=22, element='step', label='Die')
sns.histplot(Titanic[Titanic['Survived']==1]['Age'],kde=True, bins=22, element='step', label='Survived')
axes.legend()

In [ ]:
# dwa wykresy
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(14, 8))
# Mężczyźni
sns.histplot(Titanic[Titanic['Sex_Enc']==0], x='Age', hue='Survived', element='step', common_norm=False, ax=ax1, bins=22, kde=True)
ax1.set_xlabel('Wiek')
ax1.set_ylabel('Liczba przypadków')
ax1.set_title('Przeżycie mężczyzn w zależności od wieku')
legend = ax1.get_legend()
handles = legend.legend_handles
ax1.legend(handles, ['Die', 'Survived'])
# Kobiety
sns.histplot(Titanic[Titanic['Sex_Enc']==1], x='Age', hue='Survived', element='step', common_norm=False, ax=ax2, bins=22, kde=True)
ax2.set_xlabel('Wiek')
ax2.set_ylabel('CLiczba przypadków')
ax2.set_title('Przeżycie kobiet w zależności od wieku')
legend = ax2.get_legend()
handles = legend.legend_handles
ax2.legend(handles, ['Die', 'Survived'])

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
sns.violinplot(data=Titanic, x="Sex_Enc", y="Age", hue="Survived", split=True, gap=.1, inner="quart")
legend = ax.get_legend()
handles = legend.legend_handles
ax.legend(handles, ['Die', 'Survived'])
ax.set_xticklabels(['Mężczyzni', 'Kobiety'])
ax.set_xlabel('Płeć')

Pamiętajmy, że wartość naosi odciętych dla violonplota nie wskazuje częstotoliwości występowania danego zdarzenia. W ziązku z czym kluczowa analiza powinna opierać się głownie na porównaniu krzywej w maksimum. 


### Fare/Calss

In [ ]:
sns.regplot(data=Titanic, x="Pclass", y="Fare")

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(14, 8))

# Create histograms for fares (loop over first row)
for i in range(3):
    sns.histplot(Titanic[Titanic['Pclass']==(i+1)], x='Fare', hue='Survived', element='step', common_norm=False, ax=axes[0, i], bins=20, kde=True)
    legend = axes[0,i].get_legend()
    handles = legend.legend_handles
    axes[0, i].legend(handles, ['Die', 'Survived'], title = f'Pclass {3-i}')
# Create histograms for age (loop over second row)
for i in range(3):
    sns.histplot(Titanic[Titanic['Pclass']==(i+1)], x='Age', hue='Survived', element='step', common_norm=False, ax=axes[1, i], bins=20, kde=True)
    legend = axes[1,i].get_legend()
    handles = legend.legend_handles
    axes[1, i].legend(handles, ['Die', 'Survived'], title = f'Pclass {3-i}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(14, 8))
sns.violinplot(ax=ax1, data=Titanic, x="Pclass", y="Age", hue="Survived", split=True, gap=.1, inner="quart")
ax1.set_title('Violin Plot - Age by Pclass and Survived')
legend = ax1.get_legend()
handles = legend.legend_handles
ax1.legend(handles, ['Die', 'Survived'])

# Box plot on the second subplot (axes[1])
sns.boxplot(ax=ax2, data=Titanic, x="Pclass", y="Age", hue="Survived")
ax2.set_title('Box Plot - Age by Pclass and Survived')
legend = ax2.get_legend()
handles = legend.legend_handles
ax2.legend(handles, ['Die', 'Survived'])


### Rodzina

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(14, 8))
Titanic['FamilySize'] = Titanic['SibSp'] + Titanic['Parch']  # Calculate family size

sns.barplot(data=Titanic, x='FamilySize', y='Survived', palette="icefire", ax=ax1)
sns.countplot(data=Titanic, x='FamilySize', hue='Survived',  ax=ax2)
sns.countplot(data=Titanic, x='Title_Enc', hue='Survived',  ax=ax3)

### Zaokrętowanie  

## Modele

In [ ]:
#Biblioteki
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics         import make_scorer, f1_score, accuracy_score

from sklearn.neural_network  import MLPClassifier

from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import AdaBoostClassifier
from sklearn.ensemble        import GradientBoostingClassifier
from sklearn.ensemble        import HistGradientBoostingClassifier

from sklearn.naive_bayes     import GaussianNB

from sklearn.svm             import SVC

from sklearn.neighbors       import KNeighborsClassifier




In [ ]:
#Baza danych do nauki
X = Titanic.drop('Survived',axis=1)
y = Titanic['Survived']
y_test = test

In [ ]:
#Modele które zostaną wykorzystane
classifiers = [['Random Forest',RandomForestClassifier()],
               ['AdaBoost',AdaBoostClassifier()],
               ['Gradient Boosting',GradientBoostingClassifier()],
               ['Support Vector Machine', SVC()], 
               ['HGBC',HistGradientBoostingClassifier()], 
               ['Naive Bayes',GaussianNB()]
              ]
               


In [ ]:
# Ocena wydajności modeli
for metric in ['accuracy','f1']:
    SKfold = StratifiedKFold(n_splits=6)
    score_mean = []
    std = []
    
    for model in classifiers:
        clf = model[1]
        cv_result = cross_val_score(clf,X,y, cv = SKfold, scoring = metric)
        cv_result = cv_result
        score_mean.append(cv_result.mean())
        std.append(cv_result.std())
        
    models_evaluation = pd.DataFrame({metric: score_mean}, index=[i[0] for i in classifiers])
    print(models_evaluation.sort_values(metric, ascending=False))
    print('*'*32)

### Fine tuning

In [ ]:
# from sklearn.metrics import accuracy_score, f1_score, make_scorer
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.model_selection import HalvingGridSearchCV
# # Define parameter grids for each classifier
# param_grids = {
#     'Random Forest': {
#         'n_estimators': [150, 175, 200, 225, 250, 300],
#         'max_depth': [3, 4, 5, 6, 7]
#     },
#     'AdaBoost': {
#         'n_estimators': [150, 175, 200, 225, 250],
#         'learning_rate': [0.75, 0.85, 0.95, 1.0]
#     },
#     'Gradient Boosting': {
#         'n_estimators': [75, 100, 125, 150, 200],
#         'learning_rate': [0.1, 0.2, 0.25, 0.3, 0.35]
#     },
#     'Support Vector Machine': {
#         'C': [0.025, 0.05, 0.75, 0.1, 0.15, 0.25],
#         'kernel': ['linear', 'rbf']
#     },
#     'MLPC': {
#         'max_iter': [8000],
#         'solver' : ['adam'],
#         'learning_rate_init' : [0.0175, 0.02, 0.0225, 0.025, 0.03],
#         'alpha':                [0.0001, 0.00015, 0.0002, 0.00025],
#         'hidden_layer_sizes':  [(12,12), (8,8), (6,6), (3,3,3), (7,7,7) ],
#         'activation': ['relu', 'tanh']
        
#     },
#     'HGBC': {
#         'learning_rate': [0.05, 0.1, 0.25, 0.5, 1.0]
#     }
# }

# classifiers = [
#     ('Random Forest', RandomForestClassifier()),
#     ('AdaBoost', AdaBoostClassifier()),
#     ('Gradient Boosting', GradientBoostingClassifier()),
#     ('Support Vector Machine', SVC()),
#     ('MLPC', MLPClassifier()),
#     ('HGBC', HistGradientBoostingClassifier())
    
# ]



# # Perform GridSearchCV for each classifier with accuracy evaluation
# for name, clf in classifiers:
#     # Create a scoring function (using accuracy_score)
#     param_grid = param_grids[name]
#     # 1. Refit with a specific metric (e.g., 'accuracy')
#     scoring = {'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score)}
#     grid_search = GridSearchCV(estimator=clf, param_grid=param_grids[name], cv=SKfold, scoring=scoring, refit='accuracy')

#     # 2. No refitting (evaluate across all metrics)
#     # scoring = {'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score)}
#     # grid_search = GridSearchCV(estimator=clf, param_grid=param_grids[name], cv=5, scoring=scoring, refit=False)
#     #grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring=scoring)  # Adjust cv as needed
    
#     grid_search.fit(X, y)

#     # Print the best model's accuracy and parameters
#     best_model = grid_search.best_estimator_
#     best_score = grid_search.best_score_
#     best_params = grid_search.best_params_

#     print(f"\n{name} Model:")
#     print(f"Best Accuracy on Training Data: {best_score:.4f}")
#     print(f"Best Parameters: {best_params}")

#     # Evaluate the best model on the test set
#     #y_pred = best_model.predict(X_test)
#     # test_accuracy = accuracy_score(y_test, y_pred)
#     #print(f"Accuracy on Test Data: {test_accuracy:.4f}")

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures
# # Wybierz parametry
# features = ['Sex_Enc', 'FamilySize']

# # Utwórz obiekt PolynomialFeatures
# poly = PolynomialFeatures(degree=2, interaction_only=True)

# # Przekształć dane
# X_fare_sex = poly.fit_transform(Titanic[features])

# # Dodaj kolumny z nowymi cechami do DataFrame
# Titanic['Fare_Sex_Poly2'] = X_fare_sex[:, 1]
# Titanic['Fare_Sex_Poly3'] = X_fare_sex[:, 2]

# # Wyświetl pierwsze 5 wierszy DataFrame
# fig, ax = plt.subplots(figsize=(9, 9))
# sns.heatmap(Titanic.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#             square=True, linecolor='white', annot=True, cmap= 'icefire', fmt=".2f")
# ax.set_title('Pearson Correlation of Features', y=1.05, size=15)
# Titanic.head(20)

## Weryfikacja modeli

In [ ]:
Titanic

In [ ]:
test

   # Evaluate accuracy on the test set
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Print results for each model
    print(f"Model: {name}")
    print(f"Best Parameters: {best_params}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print('-' * 32)

## Kod zapasowy w markdown więc nie będzie się wyświetlać poprawnie

---------------------------